In [1]:
import torch
from src.util import make_faces

def vertex_tris(faces):
    res = []
    for vid in range(faces.max()+1):
        vertex_faces = []
        for fid, face in enumerate(faces):
            if vid in face:
                vertex_faces.append(fid)
        res.append(vertex_faces)
    return res

def vertex_tri_maps(faces):
    vts = vertex_tris(faces)
    r, c = len(vts), max([len(x) for  x in vts])
    vert_tri_indices = torch.zeros(r, c, dtype=torch.long)
    vert_tri_weights = torch.zeros(r, c)    
    for r, tris in enumerate(vts):
        if r % 1000 == 0:
            print(r, len(vts))
        weight = 1. / len(tris)
        for c, tri_id in enumerate(tris):
            vert_tri_indices[r, c] = tri_id
            vert_tri_weights[r, c] = weight
    return vert_tri_indices, vert_tri_weights.unsqueeze(dim=-1)[None]

def get_face_normals(vrt, faces):    
    v1 = vrt.index_select(1,faces[:, 1]) - vrt.index_select(1, faces[:, 0])
    v2 = vrt.index_select(1,faces[:, 2]) - vrt.index_select(1, faces[:, 0])
    face_normals = F.normalize(v1.cross(v2), p=2, dim=-1)  # [F, 3]
    return face_normals

def get_vertex_normals(face_normals, vert_tri_indices, vert_tri_weights):
    bs = face_normals.size(0)
    r, c = vert_tri_indices.shape
    fn_group = face_normals.index_select(1, 
        vert_tri_indices.flatten()).reshape(bs, r, c, 3)
    weighted_fn_group = fn_group * vert_tri_weights    
    vertex_normals = weighted_fn_group.sum(dim=-2)
    return F.normalize(vertex_normals, p=2, dim=-1)

In [2]:
from src.config import get_parser

opt = get_parser().parse_args(args=[])
opt.raster_patch_size
#vert_tri_indices, vert_tri_weights = vertex_tri_maps(faces)

256

In [3]:
faces = make_faces(opt.raster_patch_size, opt.raster_patch_size)
faces.shape

(130050, 3)

In [4]:
def vertex_tris(faces):
    res = []
    for vid in range(faces.max()+1):
        if vid % 1000 == 0:
            print(vid, faces.max()+1)
        vertex_faces = []
        for fid, face in enumerate(faces):
            if vid in face:
                vertex_faces.append(fid)
        res.append(vertex_faces)
    return res

faces = make_faces(opt.raster_patch_size, opt.raster_patch_size)
vert_tri_indices, vert_tri_weights = vertex_tri_maps(faces)

0 65536
1000 65536
2000 65536
3000 65536
4000 65536
5000 65536
6000 65536
7000 65536
8000 65536
9000 65536
10000 65536
11000 65536
12000 65536
13000 65536
14000 65536
15000 65536
16000 65536
17000 65536
18000 65536
19000 65536
20000 65536
21000 65536
22000 65536
23000 65536
24000 65536
25000 65536
26000 65536
27000 65536
28000 65536
29000 65536
30000 65536
31000 65536
32000 65536
33000 65536
34000 65536
35000 65536
36000 65536
37000 65536
38000 65536
39000 65536
40000 65536
41000 65536
42000 65536
43000 65536
44000 65536
45000 65536
46000 65536
47000 65536
48000 65536
49000 65536
50000 65536
51000 65536
52000 65536
53000 65536
54000 65536
55000 65536
56000 65536
57000 65536
58000 65536
59000 65536
60000 65536
61000 65536
62000 65536
63000 65536
64000 65536
65000 65536
0 65536
1000 65536
2000 65536
3000 65536
4000 65536
5000 65536
6000 65536
7000 65536
8000 65536
9000 65536
10000 65536
11000 65536
12000 65536
13000 65536
14000 65536
15000 65536
16000 65536
17000 65536
18000 65536
19000 

array([[ 0,  1,  4],
       [ 1,  5,  4],
       [ 1,  2,  5],
       [ 2,  6,  5],
       [ 2,  3,  6],
       [ 3,  7,  6],
       [ 4,  5,  8],
       [ 5,  9,  8],
       [ 5,  6,  9],
       [ 6, 10,  9],
       [ 6,  7, 10],
       [ 7, 11, 10],
       [ 8,  9, 12],
       [ 9, 13, 12],
       [ 9, 10, 13],
       [10, 14, 13],
       [10, 11, 14],
       [11, 15, 14]])

In [8]:
vert_tri_indices.shape, vert_tri_weights.shape

(torch.Size([16384, 6]), torch.Size([1, 16384, 6, 1]))

In [ ]:
import torch 
from collections import OrderedDict

opt = get_parser().parse_args(args=[])


opt.raster_patch_size = raster_patch_size
faces = make_faces(opt.raster_patch_size, opt.raster_patch_size)

loaded =  torch.load(os.path.join(opt.data_dir, 
            'vert_tri_{}.pth'.format(opt.raster_patch_size)))
obj = OrderedDict(OrderedDict([
  ('vert_tri_indices', vert_tri_indices),
  ('vert_tri_weights', vert_tri_weights),
  ('faces', torch.tensor(faces[None])),
])
file = './data/trimap_{}.pth'.format(opt.raster_patch_size)
torch.save(obj, file)

In [9]:
from collections import OrderedDict

obj = OrderedDict(OrderedDict([
  ('vert_tri_indices', vert_tri_indices),
  ('vert_tri_weights', vert_tri_weights),
  ('faces', torch.tensor(faces[None])),
]))
file = './data/vert_tri_{}.pth'.format(opt.raster_patch_size)
torch.save(obj, file)
file

'./data/vert_tri_128.pth'

In [10]:
loaded = torch.load(file)
loaded

OrderedDict([('vert_tri_indices',
              tensor([[    0,     0,     0,     0,     0,     0],
                      [    0,     1,     2,     0,     0,     0],
                      [    2,     3,     4,     0,     0,     0],
                      ...,
                      [32253, 32254, 32255,     0,     0,     0],
                      [32255, 32256, 32257,     0,     0,     0],
                      [32257,     0,     0,     0,     0,     0]])),
             ('vert_tri_weights',
              tensor([[[[1.0000],
                        [0.0000],
                        [0.0000],
                        [0.0000],
                        [0.0000],
                        [0.0000]],
              
                       [[0.3333],
                        [0.3333],
                        [0.3333],
                        [0.0000],
                        [0.0000],
                        [0.0000]],
              
                       [[0.3333],
                        [0.3333]

In [11]:
loaded['vert_tri_indices'].shape, loaded['vert_tri_weights'].shape

(torch.Size([16384, 6]), torch.Size([1, 16384, 6, 1]))

In [ ]:
import numpy as np

def make_faces(w, h):
    mesh_indices = []
    for iw  in range(w-1):
        for ih in range(h-1):
            mesh_indices.append([iw*w+ih, iw*w+ih+1, (iw+1)*w+ih])            
            mesh_indices.append([iw*w+ih+1, (iw+1)*w+ih+1, (iw+1)*w+ih])
    return np.array(mesh_indices)

faces = make_faces(4, 4)
faces